In [12]:
#예제
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [17]:
from langchain_openai import ChatOpenAI

openai_api_key = 'your_api_key'

model = ChatOpenAI(model = 'gpt-3.5-turbo-0125',openai_api_key=openai_api_key)


# 💎 LCEL

In [28]:
chain = prompt | model

In [29]:
chain.invoke({"country":"대한민국"}) #invoke 프롬프트와, 모델을 사슬로 엮어서 invoke 해준다.

AIMessage(content='대한민국의 수도는 서울입니다.')

## RunnablePassthrough

In [30]:
chain.invoke("미국")
#  key와 value를 매칭이되는 형태인데 현재 value만 들어왔기 때문이다. 그 오류를 개선해주는 방법


TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.

In [ ]:
chain_2 = {"country": RunnablePassthrough()} | prompt | model
chain_2.invoke("미얀마")

AIMessage(content='미얀마의 수도는 네피도입니다.')

In [31]:
#RAG에서의 예제
Chain = {"document":retriver, "question": RunnablePassthrough()} | prompt | model

prompt = "{retriever}에서 {question}에 대한 답변을 찾아주세요."

#이러한 형태로 RAG 파이프라인을 구축해서 리트리버와 모델을 연결해줄 수 있다.

NameError: name 'retriver' is not defined

## RunnableParallel  
코드를 병렬로 처리해줄때 사용

In [32]:
prompt1 = PromptTemplate.from_template("{country}의 수도는 어디인가요?")
prompt2 = PromptTemplate.from_template("{country}의 인구는 몇명이야")

In [33]:
chain_1 = {"country": RunnablePassthrough()} | prompt1 | model
chain_2 = {"country": RunnablePassthrough()} | prompt2 | model

In [35]:
map_chain = RunnableParallel(a=chain_1, b=chain_2) #Chain 을 여러개 만들어서 묶어준다.

In [36]:
map_chain.invoke({"country":"대한민국"})

{'a': AIMessage(content='대한민국의 수도는 서울입니다.'),
 'b': AIMessage(content='대한민국의 인구는 약 51,780만 명입니다.')}

## RunnableLambda
내가 만든 함수를 넣어서 파이프라인을 연결한다.

In [37]:
def combine_text(text):
    return text['a'].content + ' ' +text['b'].content


In [39]:
final_chain = map_chain | {"info": RunnableLambda(combine_text)} | PromptTemplate.from_template(
    "{info} 의 내용을 자연스럽게 교정해주고 이모티콘을 넣어서 완성도 있게 출력하세요") 

In [40]:
final_chain.invoke({"country":"대한민국"})

StringPromptValue(text='서울입니다. 대한민국의 인구는 약 5천만 명 정도입니다. 의 내용을 자연스럽게 교정해주고 이모티콘을 넣어서 완성도 있게 출력하세요')

In [41]:
final_chain = map_chain | {"info": RunnableLambda(combine_text)} | PromptTemplate.from_template(
    "{info} 의 내용을 자연스럽게 교정해주고 이모티콘을 넣어서 완성도 있게 출력하세요") | model
final_chain.invoke("미국")

AIMessage(content='미국의 수도는 워싱턴 D.C.입니다. 2021년 기준으로 미국의 인구는 약 3억 3천만 명 정도입니다. 🙂')

## Output Parser
모델의 출력을 처리하고, 그 결과를 원하는 형식으로 변환하는 역할을 합니다

### CommaSeparatedListOutputParser

In [42]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)


Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [43]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0,openai_api_key=openai_api_key)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})
# CommaSeparatedListOutputParser를 사용해서 , 를 사용해서 리스트를 만들어준다.

['Bibimbap', 'Kimchi', 'Bulgogi', 'Japchae', 'Tteokbokki']

### JsonOutputParser

In [44]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```

In [45]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)


input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [46]:
chain = prompt | model | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'To cook Bibimbap, start by preparing steamed rice. Then, sauté various vegetables such as spinach, carrots, bean sprouts, and mushrooms separately. Cook marinated beef and fry eggs sunny-side up. Place the rice in a bowl, arrange the vegetables and beef on top, and add a fried egg. Serve with spicy gochujang sauce and mix everything together before eating.'}

## Stream
Chatgpt와 Gemini 를 보면 확연히 차이가 있다.  
⌨️ Stream 은 Chatgpt 처럼 토큰단위의 처리를 할 수 있게 끔 (탁탁 타다닥 마치 타자 치듯이)
- ChatGPT : 토큰단위의 처리 후 Print  
- Gemini : 모든 토큰을 처리 후 Print  


In [49]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#invoke

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo",openai_api_key=openai_api_key)
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

'Why did the ice cream truck break down?\nIt had too many "scoops" on board!'

In [50]:
#stream
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)
#토큰 단위로 출력
    

Why did the ice cream break up with the waffle cone? It couldn't handle the rocky road!

In [51]:
#tip time 을 통해서 출력을 지연시켜서 느리게끔도 출력이 가능하다.
import time
for chunk in chain.stream("ice cream"):
    time.sleep(1)
    print(chunk, end="", flush=True)

Why did the ice cream truck break down? It had too many "scoops"!

## Batch
Input_variable 에 다양한 값을 넣고 병렬 처리를 하고 싶을때 쓸 수 있다.

In [52]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream truck break down?\n\nBecause it had too many "scoops"!',
 'Why did the spaghetti go to the party? Because it heard it was going to be a pasta-tively good time!',
 'Why did the dumpling go to the party? Because it was on a roll!']

## Async
비동기 처리 버전이 필요한 경우 사용

In [55]:
#LCEL 이 아닌경우
'''
async_client = openai.AsyncOpenAI()

async def acall_chat_model(messages: List[dict]) -> str:
    response = await async_client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

async def ainvoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return await acall_chat_model(messages)
'''

'\nasync_client = openai.AsyncOpenAI()\n\nasync def acall_chat_model(messages: List[dict]) -> str:\n    response = await async_client.chat.completions.create(\n        model="gpt-3.5-turbo", \n        messages=messages,\n    )\n    return response.choices[0].message.content\n\nasync def ainvoke_chain(topic: str) -> str:\n    prompt_value = prompt_template.format(topic=topic)\n    messages = [{"role": "user", "content": prompt_value}]\n    return await acall_chat_model(messages)\n'

In [54]:
chain.ainvoke("ice cream")

/var/folders/d4/56xjd1v51xq5ybs0jdmq217h0000gn/T/ipykernel_10576/1377747015.py:1: RuntimeWarning: coroutine 'RunnableSequence.ainvoke' was never awaited
  chain.ainvoke("ice cream")


'Why did the ice cream truck break down? \n\nIt had too many "scoops"!'

## LLM instead of chat model

In [57]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct",openai_api_key=openai_api_key)
llm_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | llm
    | output_parser
)

llm_chain.invoke("ice cream")

'\n\nWhy did the ice cream go to therapy?\n\nBecause it had a rocky road!'

## Different model provider

In [ ]:
from langchain_anthropic import ChatAnthropic
#openai 대신에 anthropic을 사용해서 사용할 수 있다.
#openai 모델 호출과 방법은 동일하기 때문에, 모델만 바꿔서 선언해주면 되기 때문에 편리하다.
anthropic = ChatAnthropic(model="claude-2")
anthropic_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | anthropic
    | output_parser
)

anthropic_chain.invoke("ice cream")

## Runtime configurability

In [58]:
from langchain_core.runnables import ConfigurableField


configurable_model = model.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="chat_openai", 
    openai=llm,
    anthropic=anthropic,
)
configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

NameError: name 'anthropic' is not defined

In [ ]:
configurable_chain.invoke(
    "ice cream", 
    config={"model": "openai"} #모델을 openai로 설정
)
stream = configurable_chain.stream(
    "ice cream", 
    config={"model": "anthropic"} #모델을 anthropic으로 설정
)
for chunk in stream:
    print(chunk, end="", flush=True) # anthropic으로 출력된다.

configurable_chain.batch(["ice cream", "spaghetti", "dumplings"]) #openai로 설정되어있기 때문에 openai로 출력된다.(default)

# await configurable_chain.ainvoke("ice cream") 비동기로 사용할 수 있다.

## Logging

https://smith.langchain.com/public/e4de52f8-bcd9-4732-b950-deee4b04e313/r

In [59]:
#랭체인 공식홈페이지를 가입해서 LangSmith를 사용할 수 있다. API key를 발급받아서 사용할 수 있다.
import os

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

anthropic_chain.invoke("ice cream")

NameError: name 'anthropic_chain' is not defined

## Fallbacks
예비 로직을 실행시켜서 실행 할 수 있다.

In [ ]:
fallback_chain = chain.with_fallbacks([anthropic_chain])

fallback_chain.invoke("ice cream")
# await fallback_chain.ainvoke("ice cream")
fallback_chain.batch(["ice cream", "spaghetti", "dumplings"])

## Full code comparison

In [ ]:
import os

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, ConfigurableField

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true" #langsmith를 통해 logging

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"    # prompt template
)
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")  # 모델선언
openai = OpenAI(model="gpt-3.5-turbo-instruct")
anthropic = ChatAnthropic(model="claude-2")
model = (
    chat_openai
    .with_fallbacks([anthropic]) #예비 모델을 설정해준다.anthropic으로 설정
    .configurable_alternatives( 
        ConfigurableField(id="model"), #모델 환경설정 정의를 통해서 openai모델 설정
        default_key="chat_openai",
        openai=openai,
        anthropic=anthropic,
    )
)

chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)
#chat_openai 모델을 사용하고, 예비모델로 anthropic을 사용하고, 모델환경설정을 통해서 openai모델을 사용한다.

## RAG Search Examples

In [ ]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")